In [4]:
import pandas as pd
import networkx as nx

we will be using networkx package http://networkx.readthedocs.io/en/networkx-1.11/index.html

### Load enron dataset into pandas
Link to Enron company https://en.wikipedia.org/wiki/Enron. You can find the data [here](https://drive.google.com/file/d/1R8TW3CUN5UM_EtQT923EtyCmICPX2NZ8/view?usp=sharing).

# Data cleaning and preparation

In [9]:
# we are using = as a separator so we don't mix text with columns separator
df_enron = pd.read_csv("enron.csv", sep="=")

## Task
Take only part of email address until the @ (not important for analysis) and add a column with link (concatenation of sender and receiver).

In [12]:
df_enron.head()

,Unnamed: 0,from,to,date,text,day_of_week
0,1,shelley.corman@enron.com,"jean.adams@enron.com, william.aldinger@enron.c...",20 Nov 2001 15,Please Send to ETS Gas Logistics Staff (Exclud...,Tue
1,5,frank.ermis@enron.com,mike.grigsby@enron.com,14 Feb 2001 02,no,Wed
2,6,joan.veselack@enron.com,"jwhited@columbiaenergygroup.com, chris.germany...",22 Mar 2000 08,Per my voicemail message to John.I talked to S...,Wed
3,7,sharon.crawford@enron.com,"chris.gaffney@enron.com, mark.powell@enron.com...",27 Oct 2000 04,Attached is Stikeman Elliott's derivatives upd...,Fri
4,9,chris.germany@enron.com,"victor.lamadrid@enron.com, edward.terry@enron....",25 Sep 2000 05,FOM New Power numbers.---------------------- F...,Mon


In [27]:
df_clean = pd.DataFrame(columns=['from','to','len(text)','day_of_week'])
k = 0
kk = 0
for i in df_enron.index:
    print(kk/len(df_enron)*100)
    receivers = df_enron.loc[i,'to'].split()
    for j in range(len(receivers)):
        to = receivers[j].split('@')[0]
        df_clean.loc[k,['from','to','len(text)','day_of_week']] = [df_enron.loc[i,'from'].split('@')[0],to,len(df_enron.loc[i,'text']),df_enron.loc[i,'day_of_week']]
        k += 1
    kk += 1

12320715
55.72243122501763
55.72537032682811
55.72830942863861
55.73124853044909
55.73418763225958
55.73712673407007
55.74006583588056
55.74300493769104
55.74594403950152
55.748883141312014
55.751822243122504
55.75476134493299
55.75770044674348
55.76063954855396
55.76357865036445
55.766517752174934
55.769456853985425
55.77239595579591
55.77533505760639
55.77827415941689
55.78121326122737
55.784152363037855
55.78709146484834
55.790030566658835
55.79296966846932
55.7959087702798
55.798847872090285
55.80178697390078
55.804726075711265
55.80766517752175
55.81060427933223
55.81354338114273
55.81648248295321
55.819421584763695
55.82236068657418
55.825299788384676
55.82823889019516
55.83117799200564
55.834117093816126
55.83705619562662
55.839995297437106
55.84293439924759
55.84587350105807
55.848812602868556
55.85175170467905
55.854690806489536
55.85762990830002
55.8605690101105
55.863508111921
55.86644721373148
55.869386315541966
55.87232541735246
55.87526451916295
55.87820362097343
55.88114

KeyboardInterrupt: 

In [33]:
df_clean = pd.DataFrame(columns=['from','to','len(text)','day_of_week'])
kk = 0

def receiver_list(to):
    to = to.split()
    to_lst = [x.split('@')[0] for x in to]
    return to_lst

for i in df_enron.index:
    print(kk/len(df_enron)*100)
    receivers = receiver_list(df_enron.loc[i,'to'])
    df_clean = df_clean.append(pd.DataFrame([[df_enron.loc[i,'from'].split('@')[0],x,len(df_enron.loc[i,'text']),df_enron.loc[i,'day_of_week']] for x in receivers], columns=df_clean.columns), ignore_index=True)
    kk += 1

.70526687044439
96.70820597225487
96.71114507406536
96.71408417587585
96.71702327768634
96.71996237949682
96.7229014813073
96.7258405831178
96.72877968492828
96.73171878673877
96.73465788854926
96.73759699035975
96.74053609217023
96.74347519398071
96.74641429579121
96.7493533976017
96.75229249941218
96.75523160122268
96.75817070303316
96.76110980484364
96.76404890665412
96.76698800846462
96.7699271102751
96.77286621208559
96.77580531389607
96.77874441570657
96.78168351751705
96.78462261932754
96.78756172113802
96.79050082294852
96.793439924759
96.79637902656948
96.79931812837997
96.80225723019046
96.80519633200095
96.80813543381143
96.81107453562191
96.81401363743241
96.8169527392429
96.81989184105338
96.82283094286386
96.82577004467436
96.82870914648484
96.83164824829532
96.8345873501058
96.83752645191629
96.84046555372679
96.84340465553727
96.84634375734775
96.84928285915824
96.85222196096873
96.85516106277922
96.8581001645897
96.86103926640018
96.86397836821068
96.86691747002116
96.

In [62]:
connection_count = df_clean.groupby(['from','to']).size()

In [57]:
df_clean = df_clean.astype({'len(text)':'int32'})

from           object
to             object
len(text)       int32
day_of_week    object
dtype: object

In [64]:
email_mean = df_clean.groupby(['from','to'])['len(text)'].mean()

In [76]:
email_we = df_clean[df_clean.day_of_week.isin([' Sat',' Sun'])]
email_we = email_we.groupby(['from','to']).size()
email_we

from              to             
a..roberts        #2.martin          4
                  a..martin          5
                  a..roberts         4
                  adam.stevens       4
                  andy.pace          4
                                    ..
william.kendrick  wayne.brunette     1
william.krenz     alan.aronowitz     1
                  e..haedicke        1
                  lisa.mellencamp    1
                  rob.walls          1
Length: 3624, dtype: int64

In [34]:
df_clean.to_csv('df_clean.csv',index=False)

## Task
Group by the dataset on the link level (variables `from` and `to`) and create these features for each link (this is input for the networkx package):

+ count of mails
+ count of emails over the weekend
+ average length of text

These features will be used as Edge Weights.

In [87]:
connection_count = connection_count.reset_index(level=[0,1])
connection_count.head()

,from,to,0
0,'todd'.delahoussaye,susan.bailey,1
1,a..bibi,dl-dji,1
2,a..bibi,sonya.johnson,1
3,a..connor,<.carter,1
4,a..connor,a..connor,1


In [88]:
email_mean = email_mean.reset_index(level=[0,1])
email_mean.head()

,from,to,len(text)
0,'todd'.delahoussaye,susan.bailey,251.0
1,a..bibi,dl-dji,700.0
2,a..bibi,sonya.johnson,700.0
3,a..connor,<.carter,13.0
4,a..connor,a..connor,13.0


In [89]:
email_we = email_we.reset_index(level=[0,1])
email_we.head()

,from,to,0
0,a..roberts,#2.martin,4
1,a..roberts,a..martin,5
2,a..roberts,a..roberts,4
3,a..roberts,adam.stevens,4
4,a..roberts,andy.pace,4


## SNA Analysis

## Task
Create directed graph with networkx package. use the networkx function `from_pandas_edgelist` with count of mails, count of emails over the weekend and average length of text as edge metrics. (parameter `edge_attr`)


In [83]:
import networkx as nx
from networkx import convert_matrix

In [92]:
G = nx.Graph()

G = nx.from_pandas_edgelist(connection_count, source='from', target="to", edge_attr=[0])
G = nx.from_pandas_edgelist(email_mean, source='from', target="to", edge_attr=['len(text)'])
G = nx.from_pandas_edgelist(email_we, source='from', target="to", edge_attr=[0])

## Basic analysis: graph properties
Find the number of nodes and edges, the average degree and the number of
connected components

In [95]:
print('Number of nodes',len(G.nodes()))
print('Number of edges',len(G.edges()))

Number of nodes 2423
Number of edges 3566


In [100]:
#nx.algorithms.degree_centrality(G)
nx.density(G)

0.0012153007937625011

### Degree distribution: find people who wrote directly to biggest number of people

## Node centralities - find the top 10 people for each case

#### find people who could be the fastest to reach all other people

#### find the most important people, most information goes through them

#### find people who are best connected with well connected people


## Communities

#### Cluster the users to different communities using function `k_clique_communities`

#### Compute clustering coefficients for each node - low clustering coefficients are in places, where network usually falls apart

## (Stretch) 
### Visualizations

#### visualize the network
it might be slow on our computers with all nodes. Feel free to play around with the graph using the smaller sample of the nodes